In [1]:
dataset_today = '22 May.xlsx'
dataset_pervious_day = '21 May.xlsx'


# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

### Reading Our Datasets 

In [3]:
dataset = pd.read_excel(dataset_today)
df = dataset.copy()

In [4]:
dataset = pd.read_excel(dataset_pervious_day)
df_pervious = dataset.copy()

# Cleaning Part

### Extract the year and the month columns


In [5]:
df.rename(columns = {
    'Order Ref/Branch': 'Area'
    , 'Receipt Number':'رقم الفاتوره'
    , 'Order Date':  'Date'
    ,'Order Ref/Sales Team': 'فريق المبيعات' 
    , 'Product/Barcode': 'Barcode'
    ,'Product/Product Category': 'Product Category'
    ,'Total cost': 'Cost'
    , 'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Barcode': 'Barcode'
    ,'Total cost': 'Cost'
    ,'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Sales Price': 'Sales Price'
}, inplace = True)

df_pervious.rename(columns = {
    'Order Ref/Branch': 'Area'
    , 'Receipt Number':'رقم الفاتوره'
    , 'Order Date':  'Date'
    ,'Order Ref/Sales Team': 'فريق المبيعات' 
    , 'Product/Barcode': 'Barcode'
    ,'Product/Product Category': 'Product Category'
    ,'Total cost': 'Cost'
    , 'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Barcode': 'Barcode'
    ,'Total cost': 'Cost'
    ,'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Sales Price': 'Sales Price'
}, inplace = True)

In [6]:
df['day'] = pd.to_datetime(df['Date']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['Date']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['Date']).dt.year
y = list(df['year'].dropna().unique())

### Change the directory in order to put all the excel files reports in the day folder

In [7]:
root_path = 'Desktop/'
os.mkdir(os.path.join(root_path, (str(int(d[0])) + ' ' + m[0] + ' '+str(int(y[0]))) ))
os.chdir(os.path.join(root_path, (str(int(d[0])) + ' ' + m[0] + ' '+ str(int(y[0]))) ))

### Split product cat Columns

In [8]:
df[['Cat1', 'Product origin', 'cat2', 'cat3', 'cat4']]= df['Product Category'].str.split("/",expand=True)
df.drop(columns= {'Product Category'}, inplace = True)
df_pervious[['Cat1', 'Product origin', 'cat2', 'cat3', 'cat4']]= df_pervious['Product Category'].str.split("/",expand=True)
df_pervious.drop(columns= {'Product Category'}, inplace = True)

#df_pervious[['Product origin', 'Cat1', 'cat2', 'cat3']]= df_pervious['Product Category'].str.split("/",expand=True)
#df_pervious.drop(columns= {'Product Category'}, inplace = True)



### Prepare the required Columns

In [9]:
df['discount amount'] = df['Quantity'] * (df['Unit Price'] - df['Subtotal'])
df['discount percentage'] = df['discount amount'] / df['Unit Price']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

In [10]:
df_pervious['discount amount'] =  df_pervious['Quantity'] * (df_pervious['Unit Price'] - df_pervious['Subtotal'])
df_pervious['discount percentage'] = df_pervious['discount amount'] / df_pervious['Unit Price']
df_pervious['Total discount'] = df_pervious['discount amount']  + df_pervious['discount percentage']
df_pervious['price before discount'] = df_pervious['Unit Price'] * df_pervious['Quantity']
df_pervious['Cost per unit'] = df_pervious['Cost'] * df_pervious['Quantity']
df_pervious['vat'] = df_pervious['Subtotal'] / 1.15
df_pervious['profit margin'] = df_pervious['vat'] - df_pervious['Cost'] 
df_pervious['Profit margin percentage'] = df_pervious['profit margin'] /df_pervious['vat']

### Cleaning the values of the branches to be in a better shape

In [11]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الإحساء' : 'الإحساء',
                             'مبيعات نقاط بيع راحتي الرس' : 'الرس' ,
                             'مبيعات نقاط بيع راحتي بريدة 2' : "بريدة 2",
                             'مبيعات نقاط بيع راحتي الدمام' : "الدمام",
                             'مبيعات نقاط بيع فرع بريدة 1' : 'بريدة 1',
                             'مبيعات نقاط بيع راحتي الطائف 1' : ' الطائف 1',
                              'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع راحتي حائل 1': 'حائل 1',
                             'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                             'مبيعات نقاط بيع راحتي حائل 2': 'حائل 2',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني',
                             'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل',
                             'مبيعات نقاط بيع فرع تبوك': 'فرع تبوك',
                             'مبيعات نقاط بيع راحتي الرياض': 'الرياض',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الطائف 2': 'راحتي الطائف 2',
                            'مبيعات نقاط بيع خميس مشيط': 'خميس مشيط'}, inplace = True)


In [12]:
df_pervious['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الإحساء' : 'الإحساء',
                             'مبيعات نقاط بيع راحتي الرس' : 'الرس' ,
                             'مبيعات نقاط بيع راحتي بريدة 2' : "بريدة 2",
                             'مبيعات نقاط بيع راحتي الدمام' : "الدمام",
                             'مبيعات نقاط بيع فرع بريدة 1' : 'بريدة 1',
                             'مبيعات نقاط بيع راحتي الطائف 1' : ' الطائف 1',
                              'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع راحتي حائل 1': 'حائل 1',
                             'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                             'مبيعات نقاط بيع راحتي حائل 2': 'حائل 2',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني',
                             'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل',
                             'مبيعات نقاط بيع فرع تبوك': 'فرع تبوك',
                             'مبيعات نقاط بيع راحتي الرياض': 'الرياض',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الطائف 2': 'راحتي الطائف 2', 
                            'مبيعات نقاط بيع خميس مشيط': 'خميس مشيط'}, inplace = True)

### Prepare different pages 

#### Page of Deposits

In [13]:
df_deposit = df[df['Product origin'] == 'منتجات مصروفات الشركة']
df_deposit.to_excel('Service and other things report.xlsx', sheet_name='منتجات مصروفات الشركة',  index = False)
df.loc[df_deposit.index,"Quantity"] = 1

#### Page of services and installment

In [14]:
df_services = df[df['Cat1'] == 'خدمات ']
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_services.to_excel(writer, sheet_name= 'خدمات',  index = False)


In [15]:
df_installment = df[df['Product origin'] == 'خدمات التقسيط']
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_installment.to_excel(writer, sheet_name= 'خدمات التقسيط',  index = False)
df_services =pd.concat([df_services, df_installment])

#### Get the additional discounts rows, put it in dataframe and save it as a file

In [16]:
df_all = df[df['Product origin'] == 'All']
df_addition_discount = df_all[df_all['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all.drop(df_addition_discount.index, inplace = True)
df_services = pd.concat([df_services, df_all])
df_addition_discount.to_excel('addition_discount report.xlsx',  index = False)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_18108\3686595714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(df_addition_discount.index, inplace = True)


#### Page of zero qunatity

In [17]:
df_0 = df[df['Quantity'] == 0 ]
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_0.to_excel(writer, sheet_name= 'صفر كمية',  index = False)
df.drop(df_0.index, inplace = True)

In [18]:
df_0 = df_pervious[df_pervious['Quantity'] == 0 ]
df_pervious.drop(df_0.index, inplace = True)

#### Final Total

In [19]:
df_final_total = df[df['Product'] == 'الإجمالي النهائي' ]
df.drop(df_final_total.index, inplace = True)


In [20]:
df_final_total = df_pervious[df_pervious['Product'] == 'الإجمالي النهائي' ]
df_pervious.drop(df_final_total.index, inplace = True)

In [21]:
branch = df[df['فريق المبيعات'] == 'مبيعات فرع الجملة' ]
df.drop(branch.index, inplace = True)

# Analytical Part

## Sales Report For Branches 

### Sales Worksheet

In [22]:
# Getting the sales of each branch in the company
df_sales= pd.pivot_table(df,index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
df_sales.rename(columns ={'Subtotal':'total sales'}, inplace = True)

# Getting the branch contribution of the sales of the company
df_sales['Sales Percent from the company'] = df_sales['total sales']/ df_sales['total sales'].sum()
df_sales.rename(columns = {'total sales': "إجمالي المبيعات"
                           , 'Sales Percent from the company': 'نسبة الفرع من إجمالي المبيعات' },inplace = True)

# Getting the sales of each branch in the company in the previous day
df_pervious_sales = pd.pivot_table(df_pervious,index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
df_pervious_sales.rename(columns ={'Subtotal':'إجمالي مبيعات اليوم السابق'}, inplace = True)

# Merging both the sales of the day and the sales of the previous day
df_sales = df_sales.merge(df_pervious_sales, on ='فريق المبيعات', how = 'left')


# getting the change of sales for each branch
df_sales['معدل التغير عن اليوم السابق'] = (df_sales['إجمالي المبيعات'] - df_sales['إجمالي مبيعات اليوم السابق']) / df_sales['إجمالي مبيعات اليوم السابق']
df_sales.drop(columns = {'إجمالي مبيعات اليوم السابق'}, inplace = True)

# filling the nan values for the branches that didn't sale anything
df_sales.fillna(0, inplace =  True)

# sorting the branches based on their performance
df_sales = df_sales.sort_values(by= "إجمالي المبيعات", ascending = False)
df_sales.reset_index(inplace = True)

# making a copy of the sales data in order to safely make changes 
df_sales_final = df_sales.copy()

# encoding the last column in the dataset and summarize the results of all branches 
df_sales_final.loc['الاجمالي', 'فريق المبيعات'] =  df_sales['فريق المبيعات'].count()
df_sales_final.loc['الاجمالي', 'معدل التغير عن اليوم السابق'] =(df_sales['إجمالي المبيعات'].sum() - df_pervious_sales['إجمالي مبيعات اليوم السابق'].sum())/ df_pervious_sales['إجمالي مبيعات اليوم السابق'].sum()
df_sales_final.loc['الاجمالي', 'نسبة الفرع من إجمالي المبيعات'] = df_sales['نسبة الفرع من إجمالي المبيعات'].sum()
df_sales_final.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()
df_sales_final.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()

# getting the order column
df_sales_final.reset_index(inplace = True)
df_sales_final.rename(columns = {'index': 'الترتيب'}, inplace = True)
df_sales_final.set_index('الترتيب', inplace = True)

# saving the insights in the sales report 
df_sales_final.to_excel('sales report.xlsx', sheet_name='مبيعات الفروع')

### Profit Worksheet

In [23]:
# Getting the profit of each branch in the company
df_profit= pd.pivot_table(df.drop(list(df_deposit.index)),index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
df_profit.rename(columns ={'profit margin':'total Profit'}, inplace = True)

# Getting the branch contribution of the profit of the company
df_profit['Profit Percent from the company'] = df_profit['total Profit']/ df_profit['total Profit'].sum()
df_profit.rename(columns = {'total Profit': "إجمالي الربح"
                           , 'Profit Percent from the company': 'نسبة الفرع من إجمالي الأرباح' },inplace = True)

# Getting the vat for each branch
df_margin_p= pd.pivot_table(df.drop(list(df_deposit.index)),index=['فريق المبيعات'], values=(['vat']), aggfunc = 'sum')
df_margin_p.rename(columns ={'vat':'الربح قبل الضريبة'}, inplace = True)

# Adding the vat to the profit dataframe
df_profit = df_profit.merge(df_margin_p, on ='فريق المبيعات', how = 'inner')

# getting the profit margin
df_profit['هامش الربح من المبيعات'] =df_profit['إجمالي الربح'] /  df_profit['الربح قبل الضريبة']  
df_profit.drop(columns = {'الربح قبل الضريبة'}, inplace = True)

# Getting the profit of each branch in the company in the previous day
df_pervious_profit = pd.pivot_table(df_pervious,index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
df_pervious_profit.rename(columns ={'profit margin':'إجمالي ربح اليوم السابق'}, inplace = True)

# Merging both the profit of the day and the profit of the previous day
df_profit = df_profit.merge(df_pervious_profit, on ='فريق المبيعات', how = 'left')

# getting the change of profit for each branch
df_profit['معدل التغير عن اليوم السابق'] = (df_profit['إجمالي الربح'] - df_profit['إجمالي ربح اليوم السابق']) / df_profit['إجمالي ربح اليوم السابق']
df_profit.drop(columns = {'إجمالي ربح اليوم السابق'}, inplace = True)

# filling the nan values for the branches that didn't sale anything
df_profit.fillna(0, inplace =  True)

# sorting the branches based on their performance
df_profit = df_profit.sort_values(by= "إجمالي الربح", ascending = False)
df_profit.reset_index(inplace = True)

# making a copy of the profit data in order to safely make changes 
df_profit_final = df_profit.copy()

# encoding the last column in the dataset and summarize the results of all branches 
df_profit_final.loc['الاجمالي', 'فريق المبيعات'] =  df_profit['فريق المبيعات'].count()
df_profit_final.loc['الاجمالي', 'معدل التغير عن اليوم السابق'] =(df_profit['إجمالي الربح'].sum() - df_pervious_profit['إجمالي ربح اليوم السابق'].sum())/ df_pervious_profit['إجمالي ربح اليوم السابق'].sum()
df_profit_final.loc['الاجمالي', 'هامش الربح من المبيعات'] = df_profit['إجمالي الربح'].sum() / (df_sales['إجمالي المبيعات'].sum()/1.15)
df_profit_final.loc['الاجمالي', 'نسبة الفرع من إجمالي الأرباح'] = df_profit['نسبة الفرع من إجمالي الأرباح'].sum()
df_profit_final.loc['الاجمالي', 'إجمالي الربح'] = df_profit['إجمالي الربح'].sum()

# getting the order column
df_profit_final.reset_index(inplace = True)
df_profit_final.rename(columns = {'index': 'الترتيب'}, inplace = True)
df_profit_final.set_index('الترتيب', inplace = True)

# saving the profit insights in the sales report 
with pd.ExcelWriter('sales report.xlsx', engine='openpyxl', mode='a') as writer:
    df_profit_final.to_excel(writer, sheet_name= 'اكثر الفروع ربحا')

### Worst Performance branches in terms of Profit Margin 

In [24]:
# Getting the data of branches with profit margins less than 30% 
df_bad_performance = df_profit_final[df_profit_final['هامش الربح من المبيعات'] < .30]

with pd.ExcelWriter('sales report.xlsx', engine='openpyxl', mode='a') as writer:
    df_bad_performance.to_excel(writer, sheet_name= 'اقل هامش ربح')

## Product Reports

### Drop additional discounts, services, and deposits from dataframe in order to get accurate results

In [25]:
# perparing our new dataframe that we will work on 
df2 = df.copy()

# In the Products report, we don't want any additional discounts, deposits, and services to appear. So, we will drop them
df2.drop(df_addition_discount.index, inplace = True)
df2.drop(df_services.index, errors= 'ignore', inplace = True)
df2.drop(df_deposit.index, errors= 'ignore', inplace = True)

### Extract the returned orders and save it 

In [26]:
# getting the returned orders and put it in a special dataframe
return_order = df2[df2['Quantity'] < 0]

# making a pivot to know the value of returned orders side by side with the quantity returned from each product 
try:
    most_return=pd.pivot_table(return_order, index = ['Barcode', 'Product'], values= ['Quantity', 'Subtotal'], aggfunc = 'sum').sort_values(by = ['Quantity'])
    most_return.reset_index(inplace = True)
# changing the names of the columns 
    most_return.rename(columns = {'Product': 'المنتج', 'Barcode': 'باركود',
                    'Quantity': 'عدد الوحدات المسترجعة', 
                     'Subtotal': 'إجمالي المبيعات'}, inplace = True)    
# making the barcode is the index of the returned orders  
    most_return.set_index('باركود' ,inplace = True)

# sorting the returned products data frame by the quantity returned and the subtotal of each product
    most_return = most_return.sort_values(by = ['عدد الوحدات المسترجعة', 'إجمالي المبيعات'])

# Saving the returned products data frame 
    most_return.to_excel('متراجعات.xlsx')
except:
    pass

### Most Sold And Most sales Worksheets

In [27]:
# Getting the most sold products 
df_Product_sold_units = pd.pivot_table(df2,index=['Product'], values=('Quantity'), aggfunc = 'sum')
df_Product_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)

# Getting the barcode of the products
df_Product_sold_units = df_Product_sold_units.merge(df[['Barcode', 'Product']].drop_duplicates(), how = 'inner', on = 'Product')

#### Total Sales Worksheet

In [28]:
# Getting the products according to the revenues we got from selling it
df_Product_sales = pd.pivot_table(df2,index=['Product'], values=(['Subtotal']), aggfunc = 'sum')

# Change the names of our columns
df_Product_sales.rename(columns ={'Subtotal':'Total Sales'}, inplace = True)

#### Merging the most sold with the total sales 

In [29]:
# adding the number of our sold units to our total sales column
df_Product_sold= df_Product_sold_units.merge(df_Product_sales, how = 'inner', on = 'Product').sort_values(by = ['Sold units'])

# reordering the columns for a better appearance 
df_Product_sold = df_Product_sold[['Product', 'Barcode', 'Sold units', 'Total Sales']]

In [30]:
product_units_sales = df_Product_sold.copy()
product_units_sales.rename(columns = {'Product': 'المنتج', 'Barcode': 'باركود',
                    'Sold units': 'عدد الوحدات المرتجعة', 'Total Sales': 'إجمالي المبيعات'}, inplace = True)

#### Save Top product according to its Sales

In [31]:
product_units_sales = df_Product_sold
product_units_sales.rename(columns = {'Product': 'المنتج', 'Barcode': 'باركود',
                    'Sold units': 'عدد الوحدات', 'Total Sales': 'إجمالي المبيعات'}, inplace = True)

product_units_sales.set_index('باركود', inplace = True)
product_units_sales.sort_values(by= 'إجمالي المبيعات', ascending = False).head(5).to_excel('Products report.xlsx',
                                                                                           sheet_name= 'اكثر المنتجات من حيث القيمة')

#### Save Top product according to its sold units

In [33]:
with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
    product_units_sales.sort_values(by= 'عدد الوحدات', ascending = False).head(5).to_excel(writer, sheet_name= 'اكثر المنتجات مبيعا')

#### Save most returned products 

In [34]:
try:
    with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
        most_return.head(5).to_excel(writer, sheet_name= 'اكثر المنتجات ارجاعا')
except:
    pass

#### Save least product sales according to its sold units 

In [35]:
least_products= product_units_sales[product_units_sales['عدد الوحدات'] > 0]
with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
    least_products.sort_values(by= ['عدد الوحدات', 'إجمالي المبيعات']).head(5).to_excel(writer, sheet_name= 'اقل المنتجات مبيعا')

#### Save least product according to its Sales

In [36]:
with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
    least_products[least_products['إجمالي المبيعات'] > 0].sort_values(by= ['إجمالي المبيعات']).head(5).to_excel(writer, sheet_name= 'اقل المنتجات مبيعا من حيث القيمة')

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


#### Top products accroding to its profit

In [37]:
df_Product_profit = pd.pivot_table(df2,index=['Product'], values=(['profit margin']), aggfunc = 'sum')
df_Product_profit.rename(columns ={'profit margin':'Total profit'}, inplace = True)

# adding the sold units columns in the dataframe of the profit
df_Product_profit = df_Product_sold_units.merge(df_Product_profit, how = 'inner', on = 'Product').sort_values(by = ['Sold units'])
df_Product_profit.rename(columns = {'Product': 'المنتج', 'Barcode': 'باركود',
                    'Sold units': 'عدد الوحدات المرتجعة', 'Total profit': 'إجمالي الربح'}, inplace = True)

# Setting the product and its barcode as an index to our dataframe
df_Product_profit.set_index(['باركود', 'المنتج'], inplace = True)

#saving the top profit products as a worksheet in the excel file "products reports"  
with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
    df_Product_profit.sort_values(by= 'إجمالي الربح', ascending = False).head(5).to_excel(writer, sheet_name= 'اكثر المنتجات ربحا')

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


## Service Report

#### Aggregate services based on our sold units

In [38]:
df_services_sold_units = pd.pivot_table(df_services,index=['Product'], values=(['Quantity']), aggfunc = 'sum')
df_services_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)

#add the barcodes to the products in our dataframe
df_services_sold_units = df_services_sold_units.merge(df[['Barcode', 'Product']].drop_duplicates(), how = 'inner', on = 'Product')

#### Aggregate services based on our sold units

In [39]:
df_services_sales = pd.pivot_table(df_services,index=['Product'], values=(['Subtotal']), aggfunc = 'sum')
df_services_sales.rename(columns ={'Subtotal':'Total Sales'}, inplace = True)

#### Save all products ordered by its sales

In [40]:
# adding the sold units column to the total sales column
df_service_sold= df_services_sold_units.merge(df_services_sales, how = 'inner', on = 'Product').sort_values(by = ['Sold units'])

#For better appearance
df_service_sold = df_service_sold[['Product', 'Barcode', 'Sold units', 'Total Sales']]


#renaming our columns
df_service_sold.rename(columns = {'Product': 'الخدمة', 'Barcode': 'باركود',
                    'Sold units': 'عدد مرات طلب الخدمة', 'Total Sales': 'إجمالي المبيعات'}, inplace = True)

# setting the service as an index
df_service_sold.set_index('الخدمة', inplace = True)

# saving our report
with pd.ExcelWriter('Products report.xlsx', engine='openpyxl', mode='a') as writer:
    df_service_sold.sort_values(by= 'إجمالي المبيعات', ascending = False).to_excel(writer, sheet_name= 'ااكثر الخدمات طلبا')

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


# Employees Reports

#### Perparing the dataFrame we will work with 

In [41]:
# Setting the quantity column for all additional discounts to zero
df_addition_discount.loc[ : ,'Quantity'] = 0
df3 =df.copy()
df3.loc[list(df_addition_discount.index) ,'Quantity'] = 0

# Removve the deposits and services from the dataframe
df3.drop(df_deposit.index, inplace =True, errors = 'ignore') 
df3.drop(df_services.index, inplace =True, errors = 'ignore') 


#### Removing the Websites sales

In [42]:
xx = df3[df3['فريق المبيعات'] == 'متجر راحتي الالكتروني']
df3.drop(xx.index, axis = 0, inplace = True)
xx = df3[df3['فريق المبيعات'] == 'المتجر الإلكتروني']
df3.drop(xx.index, axis = 0, inplace = True)

In [43]:
xx = df3[df3['فريق المبيعات'] == 'قصر البيان الإلكتروني']
df3.drop(xx.index, axis = 0, inplace = True)
xx = df3[df3['فريق المبيعات'] == 'الموقع تبوك']
df3.drop(xx.index, axis = 0, inplace = True)

#### Removing Branch Whatsup accounts from employees

In [44]:
df3.drop(df3[df3['Order Ref/Cashier'].isna()].index, inplace = True)
whatsup = df3[df3['Order Ref/Cashier'].str.contains('واتس')]
df3.drop(whatsup.index, inplace = True)

##### Aggregating Employees Based On Their Sold Units 

In [45]:
Employee_sold_units = pd.pivot_table(df3,index=['Order Ref/Cashier'], values=(['Quantity']), aggfunc = 'sum')
Employee_sold_units.rename(columns ={'Quantity':'Sold units'}, inplace = True)
df_Employee_sold_units= Employee_sold_units.merge(df3[['فريق المبيعات',
                                                'Order Ref/Cashier']].drop_duplicates(), on ='Order Ref/Cashier', how= 'inner')

##### solve the problem of the movement of salesmen from one branch to another at the same week

In [46]:
# Aggregating all branches each employee worked for
emp_duplicates_df = pd.DataFrame(df_Employee_sold_units.groupby('Order Ref/Cashier')['فريق المبيعات'].apply(lambda x: list(x)))
emp_duplicates_df.reset_index(inplace = True)

# Mereging the emp_duplicates dataframe with the Employee_sold_units dataframe 
emp_duplicates_df = emp_duplicates_df.merge(Employee_sold_units, on ='Order Ref/Cashier', how= 'inner')
emp_duplicates_df = emp_duplicates_df[['Order Ref/Cashier', 'Sold units', 'فريق المبيعات']]

In [47]:
list_employee_cor = []
employee_dup_index = []
employee_dup_indexes = list(df_Employee_sold_units[df_Employee_sold_units['Order Ref/Cashier'].duplicated()].index)
employee_dup_names = df_Employee_sold_units.loc[employee_dup_indexes, 'Order Ref/Cashier']

# Getting the employees that worked in more than branc from emp_duplicates dataframe
for e in employee_dup_names:
    list_employee_cor.append(emp_duplicates_df[emp_duplicates_df['Order Ref/Cashier'] == e].index[0])

# Getting the employees that worked in more than branch from df_Employee_sold_units dataframe    
for u in range(0, len(employee_dup_indexes)):
    employee_dup_index.append(employee_dup_indexes[u])    

# corecting the duplicates in the df_Employee_sold_units dataframe
for i in range(0 , len(list_employee_cor)):
    o = list_employee_cor[i]
    p = employee_dup_index[i]
    df_Employee_sold_units.loc[p, 'فريق المبيعات'] = str(emp_duplicates_df.loc[o, 'فريق المبيعات'])

# Dropping the duplicates so that our data frame becomes ready for our analysis
df_Employee_sold_units.drop_duplicates(subset = ['Order Ref/Cashier'], keep = 'last', inplace = True)

## Employees Sales Report

In [48]:
# aggregating the sales for each employee
df_Employee_sales= pd.pivot_table(df,index=['Order Ref/Cashier'],values=(['Subtotal']), aggfunc = 'sum')
df_Employee_sales.rename(columns ={'Subtotal':'total sales'}, inplace = True)

# add the sold units column for each employee and remove the duplicates by 'inner'
df_Employee_sales = df_Employee_sales.merge(df_Employee_sold_units, on= 'Order Ref/Cashier', how = 'inner')

# Ordering the columns of our dataframe for better appearance
df_Employee_sales = df_Employee_sales[['Order Ref/Cashier', 'فريق المبيعات', 'Sold units', 'total sales']]
df_Employee_sales.sort_values(by= 'total sales', ascending = False).head(10)
df_Employee_sales.rename(columns = {"Order Ref/Cashier": 'موظف المبيعات',
                          'Sold units': 'عدد الوحدات المباعة', 
                          'total sales': 'اجمالي المبيعات'}, inplace = True)

#### Save top seller employees according to sold units

In [49]:
# aggregating the sold units for each employee
Most_sold= pd.pivot_table(df3,index=['Order Ref/Cashier'], values=(['Subtotal']), aggfunc = 'sum')
Most_sold.rename(columns ={'Subtotal':'total sales'}, inplace = True)

# remove the duplicates 
Most_sold = Most_sold.merge(df_Employee_sold_units, on= 'Order Ref/Cashier', how = 'inner')


# Ordering the columns of our dataframe for better appearance
Most_sold = Most_sold[['Order Ref/Cashier', 'فريق المبيعات', 'Sold units', 'total sales']]

# Renaming the columns for better readability
Most_sold.rename(columns = {"Order Ref/Cashier": 'موظف المبيعات',
                          'Sold units': 'عدد الوحدات المباعة', 
                          'total sales': 'اجمالي المبيعات'}, inplace = True)

# Set the employees column as an index
Most_sold.set_index('موظف المبيعات', inplace = True)


# sace the top 10 seller employees accroding to their sold units
Most_sold.sort_values(by= 'عدد الوحدات المباعة', ascending = False).head(10).to_excel('Employees report.xlsx',
                                                                                    sheet_name='اكثر الموظفين بيعا')

#### Save least seller employees

In [50]:
# aggregating the returned products for each employee
most_return= pd.pivot_table(df3[df3['Quantity'] < 0],index=['Order Ref/Cashier'], values=['Subtotal', 'Quantity'], aggfunc = 'sum')
most_return = most_return.merge(df_Employee_sold_units[['Order Ref/Cashier', 'فريق المبيعات']], on= 'Order Ref/Cashier', how = 'inner')

# Rename the columns for better readability
most_return.rename(columns = {'Quantity': 'عدد الوحدات المسترجعة',                              
                              'Subtotal': 'اجمالي المبلغ',
                   'Order Ref/Cashier': 'موظف المبيعات',
                   'Sold units': 'عدد الوحدات المسترجعة'}, inplace = True)


# Set the employees column as an index
most_return.set_index('موظف المبيعات', inplace = True)

# sace the top 10 seller employees accroding to their returned products
with pd.ExcelWriter('Employees report.xlsx', engine='openpyxl', mode='a') as writer:
    most_return.sort_values(by= ['عدد الوحدات المسترجعة', 'اجمالي المبلغ'], ascending = True).head(10).to_excel(writer, sheet_name= 'اكثر الموظفين ارجاعا')

#### Save top seller employees according to amount of money Gained 

In [51]:
# sace the top 10 seller employees accroding to their sales
with pd.ExcelWriter('Employees report.xlsx', engine='openpyxl', mode='a') as writer:
    df_Employee_sales.set_index('موظف المبيعات', inplace = True)
    df_Employee_sales.sort_values(by= 'اجمالي المبيعات', ascending = False).head(10).to_excel(writer, sheet_name= 'اكثر الموظفين مبيعا من حيث القيمة')
    

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


#### Save least seller employees according to amount of money Gained 

In [52]:
# getting all the employees that sold more than one order
least_seller= df_Employee_sales[df_Employee_sales['اجمالي المبيعات'] > 0]
least_seller.reset_index(inplace = True)

# Removing the whatsup value from the dataframe
whatsup= list(least_seller[least_seller['موظف المبيعات'].str.contains('واتس')].index)
least_seller.drop(whatsup, inplace = True)
least_seller = least_seller[least_seller['عدد الوحدات المباعة']> 0]
# Set the employees column as an index
least_seller.set_index('موظف المبيعات', inplace = True)

# sace the least 10 employees accroding to their sales
with pd.ExcelWriter('Employees report.xlsx', engine='openpyxl', mode='a') as writer:
    least_seller.sort_values(by='اجمالي المبيعات').head(10).to_excel(writer, sheet_name= 'اقل الموظفين مبيعا من حيث القيمة')
    

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_18108\4026112519.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  least_seller.drop(whatsup, inplace = True)
C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


### Best Employee In Each Branch

In [53]:
#parpare the table of the whole employees allowing duplication for the salesmen who sell in two places
Best_employees = pd.pivot_table(df3,index=['فريق المبيعات','Order Ref/Cashier'], values=['Quantity'], aggfunc = 'sum')
Best_employees.reset_index(inplace = True)

# aggregating the sales for each employee
Best_employees_total = pd.pivot_table(df,index=['فريق المبيعات','Order Ref/Cashier'], values=['Subtotal'], aggfunc = 'sum')
Best_employees_total.reset_index(inplace = True)

#adding the sold units column
Best_employees = Best_employees.merge(Best_employees_total, on =['فريق المبيعات','Order Ref/Cashier'], how = 'inner')

# Renaming the columns for better readability
Best_employees.rename(columns = {'Quantity': 'عدد الوحدات',
                   'Subtotal': 'اجمالي المبيعات',
                   'Order Ref/Cashier': 'الموظف'}, inplace = True)

# perparing the data dataframe that will store the best employee in eaach branch
df_best_per_branch = pd.DataFrame()

#getting the list of branches we have
listbranches = list(Best_employees['فريق المبيعات'].unique())


# getting the top seller in each branch 
for b in listbranches:
    df_best_per_branch = pd.concat([df_best_per_branch, Best_employees[Best_employees['فريق المبيعات'] == b].set_index('الموظف').sort_values(by = 'اجمالي المبيعات', ascending = False).head(1)])

#### Save the top seller employee in each branch 

In [54]:
with pd.ExcelWriter('Employees report.xlsx', engine='openpyxl', mode='a') as writer:
    df_best_per_branch.to_excel(writer, sheet_name= 'افضل بائع في كل فرع')
    

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


### Ordering the employees acccording to their branches and their sales amount of money

In [55]:
#getting the list of branches we have
Branches = list(df3['فريق المبيعات'].unique())

# perparing the data dataframe that will store the best employee in eaach branch
df_branch_emp = pd.DataFrame()

# put the best employees in the df_branch_emp dataframe
for b in listbranches:
    df_branch_emp = pd.concat([df_branch_emp, Best_employees[Best_employees['فريق المبيعات'] == b].sort_values(by = 'اجمالي المبيعات', ascending = False)])

# aggregating the sales for each employee
total_branch_sales = pd.DataFrame(pd.pivot_table(df, index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum'))
df_best_per_branch = df_branch_emp.merge(total_branch_sales, how ='inner', on = 'فريق المبيعات')

# calculating the percent of sales column
df_best_per_branch['نسبة البيع من اجمالي المبيعات'] = df_best_per_branch['اجمالي المبيعات'] / df_best_per_branch['Subtotal'] 
df_best_per_branch.drop(columns = 'Subtotal', inplace = True )
df_best_per_branch.to_excel('Employees per Branch.xlsx')

#storing each branch employees in a worksheet in the excel file
for b in Branches:
    data = df_best_per_branch[df_best_per_branch['فريق المبيعات'] == b]
    data.set_index('الموظف', inplace = True)
    with pd.ExcelWriter('Employees per Branch.xlsx', engine='openpyxl', mode='a') as writer:
        data.sort_values(by= 'اجمالي المبيعات', ascending = False).head(10).to_excel(writer, sheet_name= b) 

## Employees Profit Report

In [56]:
# aggregating the porfit for each employee
df_Employee_profit= pd.pivot_table(df,index=['Order Ref/Cashier'], values=(['profit margin']), aggfunc = 'sum')
df_Employee_profit.rename(columns ={'profit margin':'total profit'}, inplace = True)

#adding the sold units column
df_Employee_profit = df_Employee_profit.merge(df_Employee_sold_units, on= 'Order Ref/Cashier', how = 'inner')

# Ordering the columns of our dataframe for better appearance
df_Employee_profit = df_Employee_profit[['Order Ref/Cashier', 'فريق المبيعات', 'Sold units', 'total profit']]

# Set the profit column as an index
df_Employee_profit.sort_values(by= 'total profit', ascending = False)

# Renaming the columns for better readability
df_Employee_profit.rename(columns = {"Order Ref/Cashier": 'موظف المبيعات',
                          'Sold units': 'عدد الوحدات المباعة', 
                          'total profit': 'اجمالي الربح'}, inplace = True)

#### Save top total sales


In [57]:
with pd.ExcelWriter('Employees report.xlsx', engine='openpyxl', mode='a') as writer:
    df_Employee_profit.set_index('موظف المبيعات', inplace = True)
    df_Employee_profit.sort_values(by= 'اجمالي الربح', ascending = False).head(10).to_excel(writer, sheet_name= 'اكثر الموظفين من حيث الربح')

C:\Users\Rahaty Store\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
